In [1]:
import re
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns


### Reading in the Data

In [2]:
race = 'TOR330'

In [3]:
TOR330_dem = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_dem.xlsx')

In [4]:
TOR330_dem['Bib'][(TOR330_dem['Year'] == 2022) &
               (TOR330_dem['Retired'] == 'Bosses')].nunique()

0

In [5]:
TOR330_dem.groupby( ['Year','Status1'] )['Status1' ].count()

Year  Status1                     
2021  DNF                             286
      Finished at Courmayeur          431
2022  DNF                             363
      Finished at Bosses               88
      Finished at Courmayeur          408
      Finished at Rifugio Frassati    101
2023  DNF                             585
      Finished at Courmayeur          622
2024  DNF                             563
      Finished at Courmayeur          533
Name: Status1, dtype: int64

In [6]:
lifebase_cut_offs_df = pd.read_excel(f'{race} Data/4. TOR330 Timetable Data/{race}_lifebase_cut_offs_df.xlsx')

# # # Convert integer seconds to timedelta
lifebase_cut_offs_df['Lifebase Duration'] = pd.to_timedelta(lifebase_cut_offs_df['Lifebase Duration_seconds'], unit='s')

# # # Convert integer seconds to timedelta
lifebase_cut_offs_df['Running Total Lifebase Duration'] = pd.to_timedelta(lifebase_cut_offs_df['Running Total Lifebase Duration_seconds'], unit='s')

lifebase_cut_offs_df

,Lifebase,Stage,Lifebase Distance (km),Lifebase Accumulated Distance Elevation (m),Lifebase Elevation Gain (m),Lifebase Accumulated Elevation (m),Lifebase Duration_seconds,Running Total Lifebase Duration_seconds,Lifebase Duration,Running Total Lifebase Duration
0,Valgrisenche IN,Stage 1,48.55,48.55,4339,4339,68400.0,68400,0 days 19:00:00,0 days 19:00:00
1,Valgrisenche OUT,Time Spent in Valgrisenche,0.00,48.55,0,4339,7200.0,75600,0 days 02:00:00,0 days 21:00:00
2,Cogne IN,Stage 2,55.45,104.00,4943,9282,75600.0,151200,0 days 21:00:00,1 days 18:00:00
3,Cogne OUT,Time Spent in Cogne,0.00,104.00,0,9282,7200.0,158400,0 days 02:00:00,1 days 20:00:00
4,Donnas IN,Stage 3,45.77,149.77,2768,12050,64800.0,223200,0 days 18:00:00,2 days 14:00:00
5,Donnas OUT,Time Spent in Donnas,0.00,149.77,0,12050,7200.0,230400,0 days 02:00:00,2 days 16:00:00
6,Gressoney IN,Stage 4,54.23,204.00,5933,17983,75600.0,306000,0 days 21:00:00,3 days 13:00:00
7,Gressoney OUT,Time Spent in Gressoney,0.00,204.00,0,17983,7200.0,313200,0 days 02:00:00,3 days 15:00:00
8,Valtournenche IN,Stage 5,33.62,237.62,3094,21077,64800.0,378000,0 days 18:00:00,4 days 09:00:00
9,Valtournenche OUT,Time Spent in Valtournenche,0.00,237.62,0,21077,7200.0,385200,0 days 02:00:00,4 days 11:00:00


In [7]:
checkpoints_bib_df = pd.read_excel(f'{race} Data/5. Clean Data for Data Visualisation/{race}_checkpoints_bib_df.xlsx')

In [8]:
all_aid_station_bib_df = pd.read_excel(f'TOR330 Data/5. Clean Data for Data Visualisation/TOR330_all_aid_station_bib_df.xlsx')

In [9]:
lifebase_bib_df = pd.read_excel(f'{race} Data/5. Clean Data for Data Visualisation/{race}_lifebase_bib_df.xlsx')

In [10]:
# # making duration hours 
# datasets = [checkpoints_bib_df, lifebase_bib_df, all_aid_station_bib_df,  TOR330_dem]
# for df in datasets:
#     df['Duration_hours'] = df['Duration_seconds']/ 3600 

In [11]:
all_aid_station_bib_df

,PK,Year,Race,Bib,Wave,Aid Station,Stage,Timestamp,Duration_seconds,Running Total Duration_seconds,Missed Lifebase Allocated Time
0,TOR330_2021_1,2021,TOR330,1,No Wave in 2021,Start,Stage 1,2021-09-12 10:00:00,NaN,0.0,Within Allocated Time
1,TOR330_2021_1,2021,TOR330,1,No Wave in 2021,Baite Youlaz,Stage 1,2021-09-12 11:42:18,6138.0,6138.0,Within Allocated Time
2,TOR330_2021_1,2021,TOR330,1,No Wave in 2021,La Thuile,Stage 1,2021-09-12 12:13:12,1854.0,7992.0,Within Allocated Time
3,TOR330_2021_1,2021,TOR330,1,No Wave in 2021,Rifugio Deffeyes,Stage 1,2021-09-12 13:40:09,5217.0,13209.0,Within Allocated Time
4,TOR330_2021_1,2021,TOR330,1,No Wave in 2021,Planaval,Stage 1,NaT,NaN,NaN,Within Allocated Time
...,...,...,...,...,...,...,...,...,...,...,...
198645,TOR330_2024_1575,2024,TOR330,1575,Wave2,Bosses,Stage 7,NaT,NaN,NaN,Within Allocated Time
198646,TOR330_2024_1575,2024,TOR330,1575,Wave2,Rifugio Frassati,Stage 7,NaT,NaN,NaN,Within Allocated Time
198647,TOR330_2024_1575,2024,TOR330,1575,Wave2,Pas Entre Deux Sauts,Stage 7,NaT,NaN,NaN,Within Allocated Time
198648,TOR330_2024_1575,2024,TOR330,1575,Wave2,Monte de la Saxe,Stage 7,NaT,NaN,NaN,Within Allocated Time


### Getting Average and Median time for lifebases

In [12]:
Stage1 = [ 'Start', 'Baite Youlaz', 'La Thuile', 'Rifugio Deffeyes',\
          'Planaval', 'Valgrisenche IN']
Stage2 = [ 'Valgrisenche OUT', 'Chalet Epee',\
          'Rhemes-Notre-Dame', 'Eaux Rousse', 'Rifugio Sella', 'Cogne IN']
Stage3 = [  'Cogne OUT', 'Goilles', 'Rifugio Dondena', 'Chardonney', 'Pontboset','Donnas IN']
Stage4 = [  'Donnas OUT', 'Perloz', 'Sassa', 'Rifugio Coda', \
          'Rifugio della Barma', 'Lago Chiaro', 'Col della Vecchia',\
          'Niel La Gruba', 'Loo', 'Gressoney IN']
Stage5 = [  'Gressoney OUT', 'Rifugio Alpenzu', 'Champoluc' ,\
          'Rifugio Grand Tournalin', 'Valtournenche IN']
Stage6 = [   'Valtournenche OUT', 'Rifugio Barmasse', 'Vareton',\
          'Rifugio Magià', 'Rifugio Cuney', 'Bivacco R. Clermont', 'Oyace', \
          'Bruson Arp', 'Ollomont IN']    
Stage7 = [ 'Ollomont OUT', 'Rifugio Champillon', 'Ponteille Desot',\
          'Bosses', 'Rifugio Frassati', 'Pas Entre Deux Sauts',\
          'Monte de la Saxe', 'FINISH']


stages =[ Stage1, Stage2, Stage3, Stage4, Stage5, Stage6, Stage7]
stages_str =[ 'Stage 1', 'Stage 2', 'Stage 3', 'Stage 4', 'Stage 5', 'Stage 6', 'Stage 7']
lifebase_time_spent = ['Valgrisenche OUT','Cogne OUT','Donnas OUT','Gressoney OUT','Valtournenche OUT','Ollomont OUT']


In [13]:
def creating_time_stats(df, column, category_order):
    
    df_merge = df.merge(
        TOR330_dem[['PK', 'Finish Category']],
        on=['PK'],
        how='left')


    df_merge['Duration_seconds'][df_merge['Duration_seconds']  <= 0] = np.nan

    stats_df = df_merge.groupby(['Finish Category', column])['Duration_seconds'].describe().reset_index(drop = False)


    stats_df[[ 'mean', 'std', 'min', '25%',
           '50%', '75%', 'max']] = stats_df[[ 'mean', 'std', 'min', '25%',
           '50%', '75%', 'max']].round(0)

    # Set 'Finish Category' as a categorical column with the defined order
    stats_df[column] = pd.Categorical(
        stats_df[column],
        categories=category_order,
        ordered=True)
    
    stats_df = stats_df.sort_values(by=column, ascending = True)
    
    

    for finish_category in df_merge['Finish Category'].unique():
        print(finish_category)
        stats_df.loc[
                (stats_df['Finish Category'] == finish_category), 'running_total_mean_seconds'
            ] =     stats_df.loc[
                (stats_df['Finish Category'] == finish_category), 'mean'
            ].cumsum()


        stats_df.loc[
                (stats_df['Finish Category'] == finish_category), 'running_total_median_seconds'
            ] =     stats_df.loc[
                (stats_df['Finish Category'] == finish_category), '50%'
            ].cumsum()


    stats_df = stats_df[['Finish Category', column,
            'count', 'mean', '50%', 'std', 'min', 'max', 
            'running_total_mean_seconds', 'running_total_median_seconds']]


    stats_df = stats_df.rename(columns={'count': f'Count_Finish_Category_{column}_seconds',
                                      'mean': f'Mean_Finish_Category_{column}_seconds',
                                      'std': f'STD_Finish_Category{column}t_seconds',
                                      '50%': f'Median_Finish_Category_{column}_seconds',
                                      'min': f'Min_Finish_Category_{column}_seconds', 
                                      'max': f'Max_Finish_Category_{column}_seconds'})
    stats_df = stats_df[stats_df[column] != 'Start']

    
    
    
    for stage,  stage_str in zip(stages, stages_str):
        stats_df.loc[stats_df[column].isin(stage), 'Stage'] = f'{stage_str}'
        
    for lifebase in lifebase_time_spent:
        lifebase_split = lifebase.split(' OUT')[0] 
        stats_df.loc[stats_df[column] == lifebase, 'Stage'] = f'Time Spent in {lifebase_split}'


    stats_df.to_excel(f'{race} Data/5. Clean Data for Data Visualisation/{race}_{column}_duration_mean_median_stats_df.xlsx', index = False)

#     print(stats_df[stats_df['Finish Category'] == 'Sub-130'])
    return stats_df

In [14]:
lifebase_stats_df = creating_time_stats(df = lifebase_bib_df, 
                    column = 'Lifebase', 
                    category_order = ['Start','Valgrisenche IN','Valgrisenche OUT',
                            'Cogne IN',  'Cogne OUT',
                            'Donnas IN', 'Donnas OUT', 
                            'Gressoney IN','Gressoney OUT',
                            'Valtournenche IN','Valtournenche OUT', 
                            'Ollomont IN','Ollomont OUT',
                            'FINISH'])

lifebase_stats_df

C:\Users\Karina\AppData\Local\Temp\ipykernel_19932\1727915605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['Duration_seconds'][df_merge['Duration_seconds']  <= 0] = np.nan


Sub-70
Sub-80
Sub-90
Sub-100
Sub-110
Sub-120
Sub-130
Sub-140
Sub-150
Over-150
DNF
nan


,Finish Category,Lifebase,Count_Finish_Category_Lifebase_seconds,Mean_Finish_Category_Lifebase_seconds,Median_Finish_Category_Lifebase_seconds,STD_Finish_CategoryLifebaset_seconds,Min_Finish_Category_Lifebase_seconds,Max_Finish_Category_Lifebase_seconds,running_total_mean_seconds,running_total_median_seconds,Stage
80,Sub-130,Valgrisenche IN,492.0,41154.0,40887.0,4248.0,30261.0,55736.0,41154.0,40887.0,Stage 1
136,Sub-80,Valgrisenche IN,18.0,27453.0,26828.0,1685.0,25420.0,32039.0,27453.0,26828.0,Stage 1
38,Sub-100,Valgrisenche IN,80.0,32851.0,32880.0,2535.0,25495.0,40300.0,32851.0,32880.0,Stage 1
94,Sub-140,Valgrisenche IN,338.0,44022.0,43902.0,4345.0,34456.0,58846.0,44022.0,43902.0,Stage 1
66,Sub-120,Valgrisenche IN,213.0,38545.0,38229.0,3582.0,31414.0,52726.0,38545.0,38229.0,Stage 1
...,...,...,...,...,...,...,...,...,...,...,...
32,Sub-100,FINISH,78.0,49218.0,48532.0,6098.0,30958.0,62121.0,346149.0,341325.0,Stage 7
130,Sub-80,FINISH,17.0,43381.0,42559.0,6993.0,36593.0,67163.0,272070.0,268707.0,Stage 7
88,Sub-140,FINISH,326.0,60934.0,60129.0,8401.0,41533.0,98635.0,484053.0,477857.0,Stage 7
4,DNF,FINISH,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stage 7


In [15]:
# # # Convert integer seconds to timedelta
lifebase_stats_df['Median_Finish_Category_Lifebase'] = pd.to_timedelta(lifebase_stats_df['Median_Finish_Category_Lifebase_seconds'], unit='s')


lifebase_stats_df[['Finish Category','Lifebase', 'Median_Finish_Category_Lifebase']][lifebase_stats_df['Finish Category'] == 'Sub-130']

,Finish Category,Lifebase,Median_Finish_Category_Lifebase
80,Sub-130,Valgrisenche IN,0 days 11:21:27
81,Sub-130,Valgrisenche OUT,0 days 00:58:09
70,Sub-130,Cogne IN,0 days 16:59:26
71,Sub-130,Cogne OUT,0 days 02:17:44
72,Sub-130,Donnas IN,0 days 11:59:26
73,Sub-130,Donnas OUT,0 days 02:33:02
75,Sub-130,Gressoney IN,0 days 22:26:34
76,Sub-130,Gressoney OUT,0 days 03:12:05
82,Sub-130,Valtournenche IN,0 days 11:29:32
83,Sub-130,Valtournenche OUT,0 days 03:22:20


In [16]:
checkpoints_stats_df = creating_time_stats(df = checkpoints_bib_df, 
                    column = 'Checkpoint', 
                    category_order = ['La Thuile', 'Valgrisenche IN', 'Valgrisenche OUT',
                                       'Eaux Rousse', 'Cogne IN', 'Cogne OUT', 'Donnas IN', 'Donnas OUT',
                                       'Rifugio della Barma', 'Niel La Gruba', 'Gressoney IN',
                                       'Gressoney OUT', 'Champoluc', 'Valtournenche IN',
                                       'Valtournenche OUT', 'Oyace', 'Ollomont IN', 'Ollomont OUT',
                                       'FINISH'])

checkpoints_stats_df

C:\Users\Karina\AppData\Local\Temp\ipykernel_19932\1727915605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge['Duration_seconds'][df_merge['Duration_seconds']  <= 0] = np.nan


Sub-70
Sub-80
Sub-90
Sub-100
Sub-110
Sub-120
Sub-130
Sub-140
Sub-150
Over-150
DNF
nan


,Finish Category,Checkpoint,Count_Finish_Category_Checkpoint_seconds,Mean_Finish_Category_Checkpoint_seconds,Median_Finish_Category_Checkpoint_seconds,STD_Finish_CategoryCheckpointt_seconds,Min_Finish_Category_Checkpoint_seconds,Max_Finish_Category_Checkpoint_seconds,running_total_mean_seconds,running_total_median_seconds,Stage
109,Sub-130,La Thuile,494.0,12733.0,12320.0,2188.0,8913.0,21942.0,12733.0,12320.0,Stage 1
49,Sub-100,La Thuile,78.0,10333.0,10183.0,1177.0,8291.0,17390.0,10333.0,10183.0,Stage 1
69,Sub-110,La Thuile,175.0,11283.0,10929.0,1921.0,8877.0,19770.0,11283.0,10929.0,Stage 1
89,Sub-120,La Thuile,212.0,12079.0,11706.0,2013.0,9291.0,21590.0,12079.0,11706.0,Stage 1
29,Over-150,La Thuile,17.0,18968.0,20684.0,3420.0,13981.0,23751.0,18968.0,20684.0,Stage 1
...,...,...,...,...,...,...,...,...,...,...,...
135,Sub-140,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,Sub-150,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,Sub-70,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,Sub-80,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# all_aid_station_bib_stats_df = creating_time_stats(df = all_aid_station_bib_df, 
#                     column = 'Aid Station', 
#                     category_order = ['Baite Youlaz', 'La Thuile', 'Rifugio Deffeyes',
#                                    'Planaval', 'Valgrisenche IN', 'Valgrisenche OUT', 'Chalet Epee',
#                                    'Rhemes-Notre-Dame', 'Eaux Rousse', 'Rifugio Sella', 'Cogne IN',
#                                    'Cogne OUT', 'Goilles', 'Rifugio Dondena', 'Chardonney', 'Pontboset',
#                                    'Donnas IN', 'Donnas OUT', 'Perloz', 'Sassa', 'Rifugio Coda',
#                                    'Rifugio della Barma', 'Lago Chiaro', 'Col della Vecchia',
#                                    'Niel La Gruba', 'Loo', 'Gressoney IN', 'Gressoney OUT',
#                                    'Rifugio Alpenzu', 'Champoluc', 'Rifugio Grand Tournalin',
#                                    'Valtournenche IN', 'Valtournenche OUT', 'Rifugio Barmasse', 'Vareton',
#                                    'Rifugio Magià', 'Rifugio Cuney', 'Bivacco R. Clermont', 'Oyace',
#                                    'Bruson Arp', 'Ollomont IN', 'Ollomont OUT', 'Rifugio Champillon',
#                                    'Ponteille Desot', 'Bosses', 'Rifugio Frassati', 'Pas Entre Deux Sauts',
#                                    'Monte de la Saxe', 'FINISH'])

# all_aid_station_bib_stats_df

### Who ran too easy or hard at the start?

In [18]:
# sub_TOR330_dem_bib_list = list(TOR330_dem['PK'][TOR330_dem['Finish Category'] == 'Sub-130'].unique())

# new_bib_list = []
# for bib in sub_TOR330_dem_bib_list:
#     df = lifebase_bib_df[
#                 ((lifebase_bib_df['Duration_hours']> 14) &
#                 (lifebase_bib_df['Lifebase'] == 'Valgrisenche IN') )
#     &
#                 (lifebase_bib_df['PK']== bib)
#                ]
    
#     new_bib_list.append(df)
# df= pd.concat(new_bib_list)
# pk_unique = list(df['PK'].unique())

# for pk in pk_unique:
#     print(lifebase_bib_df[['PK','Lifebase', 'Timestamp', 'Duration_hours']][(lifebase_bib_df['PK'] == pk)  ], '\n', '*'*40,  '\n',)
#     print(TOR330_dem[['PK', 'Finish Category', 'Duration_hours']][(TOR330_dem['PK'] == pk)  ], '\n','\n','\n','\n', '*'*40,  '\n',)

In [19]:
# sub_TOR330_dem_bib_list = list(TOR330_dem['PK'][TOR330_dem['Finish Category'] == 'Sub-140'].unique())

# new_bib_list = []
# for bib in sub_TOR330_dem_bib_list:
#     df = lifebase_bib_df[
#                 ((lifebase_bib_df['Duration_hours']< 10) &
#                 (lifebase_bib_df['Lifebase'] == 'Valgrisenche IN') ) &
#                 (lifebase_bib_df['PK']== bib)]
#     new_bib_list.append(df)
    
# df= pd.concat(new_bib_list)

# pk_unique = list(df['PK'].unique())

# for pk in pk_unique:
#     print(lifebase_bib_df[['PK', 'Wave', 'Lifebase', 'Duration_hours']][(lifebase_bib_df['PK'] == pk)  ], '\n', '\n',)
#     print(TOR330_dem[['PK', 'Finish Category', 'Duration_hours']][(TOR330_dem['PK'] == pk)  ], '\n', '*'*40,  '\n',)

In [20]:
# # reading in Raw Data
# races = ['TOR330']
# years = [ 
# #     '2021',
# #         '2022',
# #          '2023', 
#     '2024'
#         ]

# TORX_df = {}

# for race in races:
#     for year in years:
#         df = pd.read_excel(f'{race} Data/1. 100x100trail/{race}_{year}.xlsx',
#                                  dtype={'Start Date': 'string',
#                                         'Year': 'string'})
#         print(f'{race}_{year} {df.shape}')
#         # Store the DataFrame in the dictionary with a key like 'TOR330_2021'
#         TORX_df[f'{race}_{year}'] = df
#     print('*'*50)
    
# TORX_df_concat = pd.concat(TORX_df)
# TOR330 = TORX_df_concat[TORX_df_concat['Year'] == year]

In [21]:
# sub_TOR330_dem_bib_list = list(TOR330_dem['PK'][TOR330_dem['Finish Category'] == 'Sub-90'].unique())

# new_bib_list = []
# for bib in sub_TOR330_dem_bib_list:
#     df = lifebase_bib_df[
#                 ((lifebase_bib_df['Duration_hours']> 8) &
#                 (lifebase_bib_df['Lifebase'] == 'Valgrisenche OUT') ) &
#                 (lifebase_bib_df['PK']== bib)]
#     new_bib_list.append(df)
    
# df= pd.concat(new_bib_list)

# pk_unique = list(df['PK'].unique())

# for pk in pk_unique:
#     print(lifebase_bib_df[['PK', 'Wave', 'Lifebase', 'Duration_hours']][(lifebase_bib_df['PK'] == pk)  ], '\n', '\n',)
#     print(TOR330_dem[['PK', 'Finish Category', 'Duration_hours']][(TOR330_dem['PK'] == pk)  ], '\n', '*'*40,  '\n',)